# Hypotéza 1
Na silnicích první třídy se při nehodách umíralo se stejnou pravděpodobností jako na
silnicích třetí třídy.

K ověření hypotézy využijte $\chi^2$ test s tím, že také určíte, jestli nehody na silnicích 1. třídy vedly častěji či méně často k fatální nehodě. V tom vám může pomoct “expected” výstup $\chi^2$ testu. Nerozlišujte počet následků, ale pouze nehody, kde došlo k jednomu a více úmrtí (${p13a} > 0$).

# Hypotéza 2
Při nehodách vozidel značky Škoda je škoda na vozidle nižší než při nehodách vozidel Audi.

Využijte vhodný test, který vyberete na základě distribuce dat, nevyužívejte $\chi^2$ test. Určete, zda je škoda na vozidlech vyšší či nižší a zda je tato vlastnost na požadované hladině významnosti.